In [ ]:
using Piccolo
using LinearAlgebra
using CairoMakie

const ⊗ = kron

# I. Rotations
-----

## Goals
- Geodesics
- Controllability of LTI systems
- Dynamical Lie algebras and reachability

_Comparing initialization and optimization when restricted to a feasible manifold, and how warm starts are enabled by direct control._

![Initialization comparison between direct and indirect optimization.](../assets/fig2_feas-manifold.png)

In [ ]:
U_goal = PAULIS.X
T = 50
Δt = 0.2

# U_goal = exp(-im * H_eff * T * Δt)
H_eff = im * log(U_goal / T / Δt)

In [ ]:
# Draw the geodesic.

## Controllability

- Quick check: What happens when we kick a system $x_{n+1} = A x_n + B u_n$?

\begin{equation}
\mathcal{C} = \begin{bmatrix}
    B & A B & A^2 B & \cdots & A^{n-1} B
\end{bmatrix}
\end{equation}

- Quick check: Why did we stop at $n-1$?

**Example**
- Test on a linear system in 2D. Recall our $F = ma$ system.

In [ ]:
function continuous_to_discrete(A, B, h)
    # Construct augmented matrix for matrix exponential
    augmented_matrix = [
        A B; 
        zeros(size(B, 2), size(A, 1)) zeros(size(B, 2), size(B, 2))
    ]

    # Compute matrix exponential
    exp_matrix = exp(augmented_matrix * h)

    # Extract discrete LTI system matrices
    A_h = exp_matrix[1:size(A, 1), 1:size(A, 2)]
    B_h = exp_matrix[1:size(A, 1), size(A, 2)+1:end]

    return A_h, B_h
end

# Extract discrete LTI system matrices
A_cts = [0.0 1.0; -1.0 -0.1]
B_cts = [0.0; 1.0]
h = 0.1  # Time step
A, B = continuous_to_discrete(A_cts, B_cts, h)

- Let's create a 2D state.
\begin{equation}
    z = \begin{bmatrix} x \\ \dot{x} \\ y \\ \dot{y} \end{bmatrix}
\end{equation}

In [ ]:
Axy = I(2) ⊗ A
Bxy = [B zeros(2); zeros(2) B]

C = hcat([Axy^n * Bxy for n in 0:size(Axy, 1)-1]...)

In [ ]:
rank(C)

In [ ]:
Axy = I(2) ⊗ A

# Directly move the X particle only
Bxy = [[1; 0] zeros(2); [0; 1] zeros(2)]

C = hcat([Axy^n * Bxy for n in 0:size(Axy, 1)-1]...)

In [ ]:
rank(C)

What about quantum systems? They are nonlinear.

- BCH
\begin{equation}
    e^{X} e^{Y} = e^{X + Y + \tfrac{1}{2} [X, Y] + \tfrac{1}{12} ([X, [X, Y]] + [Y, [Y, X]])} 
\end{equation}

- Collect commutators, forming the **Dynamical Lie algebra**.

- Quick check: How can we test for linear dependence?

In [ ]:
# Linearly dependent: QR decomposition has a zero on diagonal 
H_drives = [PAULIS.X, PAULIS.Y]
# H_drives = [PAULIS.X, PAULIS.X]

M = stack(vec.(H_drives))
qr(M).R

- Quick check: What about systems with drift?

# II. Demos
-----

From the IPOPT documentation (https://coin-or.github.io/Ipopt/OUTPUT.html):

- **inf_pr**: The unscaled constraint violation at the current point. This quantity is the infinity-norm (max) of the (unscaled) constraints ( gL≤g(x)≤gU in (NLP)). During the restoration phase, this value remains the constraint violation of the original problem at the current point. The option inf_pr_output can be used to switch to the printing of a different quantity.

- **inf_du**: The scaled dual infeasibility at the current point. This quantity measure the infinity-norm (max) of the internal dual infeasibility, Eq. (4a) in the implementation paper [12], including inequality constraints reformulated using slack variables and problem scaling. During the restoration phase, this is the value of the dual infeasibility for the restoration phase problem.

- **lg(mu)**: log10 of the value of the barrier parameter μ.

- **||d||**: The infinity norm (max) of the primal step (for the original variables x and the internal slack variables s). During the restoration phase, this value includes the values of additional variables, p and n (see Eq. (30) in [12]).

- **lg(rg)**: log10 of the value of the regularization term for the Hessian of the Lagrangian in the augmented system ( δw in Eq. (26) and Section 3.1 in [12]). A dash ("-") indicates that no regularization was done.

- **alpha_du**: The stepsize for the dual variables ( αzk in Eq. (14c) in [12]).